In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'augmented-forest-segmentation:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1650618%2F2738848%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240404%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240404T110248Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D74917aa68ed4c57cdc91bf81a320f40beaed4272b89fd3d21e1701c40aedc1c0b4c9ae1a4168583b9ba8fec84ab1d9a474cb9de36dac9a351acf576cc8e7f6a7e7a3e649a963b049751477ffc7eae281fb8ca2ec6aa287a122300aec073e47ca9cf79a54f638931cb3468d2e0e8467dcd31da209447a801114c9b924f5b218c1c2da84b577b6773c6de889569ff8336bfab56f39b7428ce4cf8b609fb3b0eea9d3bc18df20b3889ae43526bf7a31d6e9d6cb460589374da89e276e944f9ae8164a5f402e606f12192dcc81ac3e5624fb193508ce701af87841d435dd5d55ecd60545f641ff871ad3283075acdf3c43e09d34b52cce30a8e469fbf64036795872'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 180628283 bytes downloaded
Downloaded and uncompressed: augmented-forest-segmentation
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/27460_sat_80.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/242583_sat_15.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/417313_sat_82.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/417313_sat_63.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/899693_sat_81.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/784518_sat_78.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/875328_sat_06.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/134465_sat_44.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/277994_sat_53.jpg
/kaggle/input/augmented

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataset import random_split
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

In [4]:
# Определение архитектуры Unet
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [5]:
# Определение кастомного датасета
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image, _ = self.data[index]  # Получение изображения
        return image


In [6]:
# Предварительная обработка данных
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Создание датасета
dataset = CustomDataset('/kaggle/input/augmented-forest-segmentation', transform=transform)

# Разделение датасета на обучающую и тестовую выборки
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Создание DataLoader для обучения и тестирования
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
# Инициализация модели и оптимизатора
model = UNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Обучение модели
device = torch.device("cpu")
model.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for i, images in enumerate(train_loader):
        images = images.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, images)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:  # выводить промежуточные результаты каждые 10 итераций
            print(f'Epoch {epoch+1}/{num_epochs}, Iteration {i}/{len(train_loader)}, Loss: {loss.item()}')

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

Epoch 1/10, Iteration 0/256, Loss: 0.6940451264381409
Epoch 1/10, Iteration 10/256, Loss: 0.5705979466438293
Epoch 1/10, Iteration 20/256, Loss: 0.33552876114845276
Epoch 1/10, Iteration 30/256, Loss: 0.40561506152153015
Epoch 1/10, Iteration 40/256, Loss: 0.2568013370037079
Epoch 1/10, Iteration 50/256, Loss: 0.34717848896980286
Epoch 1/10, Iteration 60/256, Loss: 0.3449654281139374
Epoch 1/10, Iteration 70/256, Loss: 0.3598593473434448
Epoch 1/10, Iteration 80/256, Loss: 0.29696813225746155
Epoch 1/10, Iteration 90/256, Loss: 0.30025115609169006
Epoch 1/10, Iteration 100/256, Loss: 0.26468104124069214
Epoch 1/10, Iteration 110/256, Loss: 0.25220024585723877
Epoch 1/10, Iteration 120/256, Loss: 0.3435309827327728
Epoch 1/10, Iteration 130/256, Loss: 0.22074498236179352
Epoch 1/10, Iteration 140/256, Loss: 0.3671153485774994
Epoch 1/10, Iteration 150/256, Loss: 0.2387062907218933
Epoch 1/10, Iteration 160/256, Loss: 0.23721660673618317
Epoch 1/10, Iteration 170/256, Loss: 0.43961501121

In [9]:
# Сохранение весов модели
torch.save(model.state_dict(), 'unet_model.pth')